In [1]:
# -*- coding: UTF-8 -*-
%matplotlib inline
import itertools
import os
import random
import sys
import matplotlib.pyplot as plt
import cv2
import numpy as np
import shutil # Just for directory removal

from PIL import Image, ImageDraw, ImageFont

In [2]:
# Script Variables :
FONT_HEIGHT = 30  #Charachters quality
MAX_RUNS = 100
PLATE_DIM = (120, 600)
OUTPUT_SHAPE = (128, 256)
DEFAULT_POSITION = 0
NOISE_SCALE = 0.1 # entre 0.01 et 0.1 ; Pensez a utiliser un rapport de bruit variable ;) 


FONT_DIR = "/root/jupyter/Deep_Learning/fonts/"
IMGS_DIR = '/root/jupyter/Haar_Cascade/pos/'
BGS_DIR = '/root/jupyter/Haar_Cascade/neg/'
DIRS = [IMGS_DIR]

number_fonts = {'font_1' : ImageFont.truetype(FONT_DIR + 'numbers_font_1.ttf', FONT_HEIGHT * 4)}
letter_fonts = {"font_1" :ImageFont.truetype(FONT_DIR + "letters_font_1.ttf", FONT_HEIGHT * 4)}

letters = ['أ', 'ب', 'ج', 'د', 'ش', 'س']
numbers = ['1','2','3','4','5','6','7','8','9','0']
latin_letters = {'أ':'A', 'ب':'B', 'ج':'J', 'د':'D', 'ش':'X', 'س':'S', } #Couldn't save files with arabic letters (for some reason)

In [3]:
## Creation des nombres et lettres a utiliser lors de la creation de la plaque
def bounding_box(plate_mask, DEFAULT = DEFAULT_POSITION, OUTPUT = OUTPUT_SHAPE):
    top = DEFAULT
    bottom = DEFAULT
    for i in range(0,OUTPUT[0],1):
        if top == DEFAULT and any(plate_mask[i] > 0) :
            top = i
        if bottom == DEFAULT and any(plate_mask[OUTPUT[0] - 1 - i] > 0) :
            bottom = OUTPUT[0] - 1 - i
        if top != DEFAULT and bottom != DEFAULT :
            break
    left = DEFAULT
    right = DEFAULT
    for i in range(0,OUTPUT[1],1):
        if left == DEFAULT and any(plate_mask.T[i] > 0) :
            left = i
        if right == DEFAULT and any(plate_mask.T[OUTPUT[1] - 1 - i] > 0) :
            right = OUTPUT[1] - 1 - i
        if left != DEFAULT and right != DEFAULT :
            break
    return top, left, bottom, right


letter_dim = {}
letter_img = {}
for font in letter_fonts :
    for letter in letters:
        img = np.zeros((200, 200))
        img = Image.fromarray(img)
        draw = ImageDraw.Draw(img)
        draw.text((20, 20), letter, font=letter_fonts[font])
        img = np.array(img)
        top, left, bottom, right = bounding_box(img, 0,(200, 200))
        letter_dim[font + '_' + letter] = (bottom - top, right - left)
        img = img[top:bottom,left:right]
        letter_img[font + '_' + letter] = img
        
number_dim = {}
number_img = {}
for font in number_fonts :
    for number in numbers:
        img = np.zeros((100, 47)) # it's known
        img = Image.fromarray(img)
        draw = ImageDraw.Draw(img)
        draw.text((-2,-10), number, font=number_fonts[font])
        img = np.array(img)
        top, left, bottom, right = bounding_box(img, 0, (100, 47))
        number_dim[font + '_' + number] = (100, 47)
        number_img[font + '_' + number] = img

In [4]:
##Fonctions du programme :
def random_plate():
    ''' Une fonction permettant la creation d'une plaque a valeurs aleatres qui 'flottent' sur la surface de la plaque (Dimensions) '''
    #Creation du code
    ## TODO : ajouter un nombre de motifs differents separant la lettre des autre nombres
    number = str(random.randint(1, 99999))
    letter = random.sample(letters, 1)[0]
    region = str(random.randint(1, 99))
    line_width = np.random.randint(3,8)
    line_length = np.random.randint(90, 110)
    number_font = random.sample(list(number_fonts),1)[0]
    letter_font = random.sample(list(letter_fonts),1)[0]
    
    code = '{:<5}-{:}-{:<2}'.format(number, letter, region)
    latin_code = '{:<5}-{:}-{:<2}'.format(number, latin_letters[letter], region)
    plate = np.zeros(PLATE_DIM)
    
    # Ajout des motifs conseccutive suit une loi decroissante qui tend a positionner les derniers nombre extremmement a gauche
    # Par contre, un ajout oscillatoire suit une loi gaussienne
    # Ajout des nombres a gouche
    occupied = PLATE_DIM[1] - len(number + region) * 47 - letter_dim[letter_font + '_' + letter][1] - 2 * line_width
    x_cursor = np.random.randint(0, occupied - 1)
    for n in number :
        y_cursor = np.random.randint(1, PLATE_DIM[0] - 100)
        plate[y_cursor : y_cursor + number_dim[number_font + '_' + n][0], x_cursor : x_cursor + number_dim[number_font + '_' + n][1]] = number_img[number_font + '_' + n]
        x_cursor += number_dim[number_font +  '_' + n][1]
    # Ajout des nombres a droites (La region)
    occupied = x_cursor + line_width * 2 + letter_dim[letter_font + '_' + letter][1] + len(region) * 47
    x2_cursor = np.random.randint(occupied - 1 , PLATE_DIM[1] - 1)
    for n in region :
        x2_cursor -= number_dim[number_font +  '_' + n][1]
        y_cursor = np.random.randint(0, PLATE_DIM[0] - 100)
        plate[y_cursor : y_cursor + number_dim[number_font + '_' + n][0], x2_cursor : x2_cursor + number_dim[number_font + '_' + n][1]] = number_img[number_font + '_' + n]
    
    # Ajout de La ligne a droite
    occupied = x2_cursor - letter_dim[letter_font + '_' + letter][1] - 2 * line_width
    x_cursor = np.random.randint(x_cursor - 1, occupied + 1)
    y_cursor = np.random.randint(1, PLATE_DIM[0] - line_length)
    plate[y_cursor : y_cursor + line_length, x_cursor : x_cursor + line_width] = np.ones((line_length, line_width))
    x_cursor += line_width
    
    # Ajout de la ligne a gauche
    occupied = x_cursor + letter_dim[letter_font + '_' + letter][1] + line_width
    x2_cursor = np.random.randint(occupied - 1, x2_cursor + 1)
    x2_cursor -= line_width
    y_cursor = np.random.randint(0, PLATE_DIM[0] - line_length)
    plate[y_cursor : y_cursor + line_length, x2_cursor : x2_cursor + line_width] = np.ones((line_length, line_width))
    
    # Ajout de la lettre
    occupied = x2_cursor - letter_dim[letter_font + '_' + letter][1]
    x_cursor = np.random.randint(x_cursor - 1, occupied + 1)
    y_cursor = np.random.randint(0, PLATE_DIM[0] - letter_dim[letter_font + '_' + letter][0])
    plate[y_cursor : y_cursor + letter_dim[letter_font + '_' + letter][0], x_cursor : x_cursor + letter_dim[letter_font + '_' + letter][1]] = letter_img[letter_font + '_' + letter]
    
    return plate, latin_code


def show_image(image, labels):
    '''Une fonction permettant l'affichage d'une image et le tracage d'un rectangle autour de l'objet annoté (s'il y'en a)''' 
    (p, top, left, bottom, right) = labels
    cv2.rectangle(image, (int(left), int(top)), (int(right), int(bottom)) , (0,0,0),1)
    plt.imshow(image,cmap='gray')


def save_file(img_idx, plate, p, latin, box):
    img_name = IMGS_DIR + '{:08d}.png'.format(img_idx)
    line = '{:08d}.png'.format(img_idx) + ' 1 {} {} {} {}\n'.format(box[1],box[0],box[3],box[2])
    with open('/root/jupyter/Haar_Cascade/pos.txt','a') as f:
        f.write(line)
    cv2.imwrite(img_name, plate * 255)


def matrice_rotation(yaw, pitch, roll):
    '''Une fonction permettant de generer une matrice de rotation tridimentionnelle a partir des trois angles fournies (yaw, pitch, roll)'''
    # Rotation selon l'axe Y
    c, s = np.cos(yaw), np.sin(yaw)
    M = np.matrix([   [  c, 0.,  s],
                      [ 0., 1., 0.],
                      [ -s, 0.,  c]])
    
    # Rotation selon l'axe X
    c, s = np.cos(pitch), np.sin(pitch)
    M = np.matrix([   [ 1., 0., 0.],
                      [ 0.,  c, -s],
                      [ 0.,  s,  c]]) * M
    
    # Rotation selon l'axe Z
    c, s = np.cos(roll), np.sin(roll)
    M = np.matrix([   [  c, -s, 0.],
                      [  s,  c, 0.],
                      [ 0., 0., 1.]]) * M
    return M

def pick_colors():
    '''Choix de couleurs utilisees pour la plaque et son contenu (chiffres) '''
    first = True
    while first or plate_color - text_color < 0.3:
        text_color = random.random()
        plate_color = random.random()
        if text_color > plate_color:
            text_color, plate_color = plate_color, text_color
        first = False
    return text_color, plate_color

def rounded_rect(shape, rayon):
    '''Un mask pour la plaque plus realiste (Les contours sont plutot des arcs / circles)'''
    out = np.ones(shape)
    # On ne laisse que les coins (des carreax de cotes R avec R le rayon fourni en parametre) du mask en 1(blanc)
    out[:rayon, :rayon] = 0.0
    out[-rayon:, :rayon] = 0.0
    out[:rayon, -rayon:] = 0.0
    out[-rayon:, -rayon:] = 0.0
    # On remplit les cercle de rayon R sur les quatres qoins du mask
    cv2.circle(out, (rayon, rayon), rayon, 1.0, -1)
    cv2.circle(out, (rayon, shape[0] - rayon), rayon, 1.0, -1)
    cv2.circle(out, (shape[1] - rayon, rayon), rayon, 1.0, -1)
    cv2.circle(out, (shape[1] - rayon, shape[0] - rayon), rayon, 1.0, -1)
    return out

def generate_plate(font_height):
    '''Combinaison des elements de la plaque'''
    rayon = 1 + int(font_height * 0.5 * random.random())
    text_color, plate_color = pick_colors()
    text_mask, latin_code = random_plate()
    out_shape = text_mask.shape
    plate = (np.ones(out_shape) * plate_color * (1. - text_mask) + np.ones(out_shape) * text_color * text_mask)
    mask = rounded_rect(out_shape, rayon)
    return plate, mask, latin_code

def generate_bg(num_bg_images):
    found = False
    while not found:
        # Prendre une background au hasard
        fname = BGS_DIR + '{:08d}.jpg'.format(random.randint(0, num_bg_images - 1))
        bg = cv2.imread(fname)
        bg = cv2.cvtColor(bg, cv2.COLOR_RGB2GRAY) / 255.
        # Condition generalement realisee (Les bg que j'ai choisi(opencv negatives) verifient cette cond)
        if (bg.shape[1] >= OUTPUT_SHAPE[1] and
            bg.shape[0] >= OUTPUT_SHAPE[0]):
            found = True
    x = random.randint(0, bg.shape[1] - OUTPUT_SHAPE[1])
    y = random.randint(0, bg.shape[0] - OUTPUT_SHAPE[0])
    bg = bg[y:y + OUTPUT_SHAPE[0], x:x + OUTPUT_SHAPE[1]]

    return bg

def generate_img(num_bg_images):
    bg = generate_bg(num_bg_images)

    plate, plate_mask, latin_code = generate_plate(FONT_HEIGHT)
    
    #matrice de transformation affine (trans * rot * scale) & rot, scale and trans detection limits
    M, out_of_bounds = make_affine_transform(
                            from_shape=plate.shape,
                            to_shape=bg.shape,
                            min_scale=0.3,
                            max_scale=1.2,
                            rotation_variation=0.3,
                            scale_variation=1.2,
                            translation_variation=1.5)
    
    #when only working with positive images
    if out_of_bounds:
        return generate_img(num_bg_images)
    
    plate = cv2.warpAffine(plate, M, (bg.shape[1], bg.shape[0]))
    plate_mask = cv2.warpAffine(plate_mask, M, (bg.shape[1], bg.shape[0]))
    
    #bounding box detection
    box = bounding_box(plate_mask)
    
    #Superposition des images
    out = plate * plate_mask + bg * (1.0 - plate_mask) 
    
    #some noise   
    out += np.random.normal(scale=NOISE_SCALE, size=out.shape)
    
    #Transformation des valeurs des pixels en valeurs entre 1 et 0 de l'image de sortie
    out = np.clip(out, 0., 1.)
    return out, not out_of_bounds, latin_code, box # Not out_of_bounds is equivalent to presence

def make_affine_transform(from_shape, to_shape, 
                          min_scale, max_scale,
                          scale_variation=1.0,
                          rotation_variation=1.0,
                          translation_variation=1.0):
    out_of_bounds = False

    from_size = np.array([[from_shape[1], from_shape[0]]]).T
    to_size = np.array([[to_shape[1], to_shape[0]]]).T

    scale = random.uniform((min_scale + max_scale) * 0.5 -
                           (max_scale - min_scale) * 0.5 * scale_variation,
                           (min_scale + max_scale) * 0.5 +
                           (max_scale - min_scale) * 0.5 * scale_variation)
    if scale > max_scale or scale < min_scale:
        out_of_bounds = True
    roll = random.uniform(-0.2, 0.2) * rotation_variation
    pitch = random.uniform(-0.2, 0.2) * rotation_variation
    yaw = random.uniform(-0.2, 0.2) * rotation_variation

    # Puisque on travail on deux dimensions :( 
    M = matrice_rotation(yaw, pitch, roll)[:2, :2]
    h, w = from_shape
    corners = np.matrix([[-w, +w, -w, +w],
                            [-h, -h, +h, +h]]) * 0.5
                            
    skewed_size = np.array(np.max(M * corners, axis=1) -
                              np.min(M * corners, axis=1))
    
    # Set the scale as large as possible such that the skewed and scaled shape is less than or equal to the desired ratio in either dimension.
    scale *= np.min(to_size / skewed_size)

    # Set the translation such that the skewed and scaled image falls withinthe output shape's bounds.
    trans = (np.random.random((2,1)) - 0.5) * translation_variation
    trans = ((2.0 * trans) ** 5.0) / 2.0
    if np.any(trans < -0.5) or np.any(trans > 0.5):
        out_of_bounds = True
    trans = (to_size - skewed_size * scale) * trans

    center_to = to_size / 2.
    center_from = from_size / 2.

    M *= scale
    M = np.hstack([M, trans + center_to - M * center_from])

    return M, out_of_bounds
    

def generate_imgs():
    num_bg_images = len(os.listdir(BGS_DIR))
    while True:
        yield generate_img(num_bg_images)



In [6]:
##Programme principal :
if __name__ == '__main__' :
    for DIR in DIRS:
        shutil.rmtree(DIR, ignore_errors=True)
        os.makedirs(DIR)
    im_gen = itertools.islice(generate_imgs(),MAX_RUNS)
    for img_idx, (plate, p, latin, box) in enumerate(im_gen):
        save_file(img_idx, plate, p, latin, box)
        #show_image(plate, [p] + list(box))
        i = (img_idx / MAX_RUNS) * 100
        if int(i) == i:
            sys.stdout.write("\r%d%%" % i)
            sys.stdout.flush()
    print('\rDONE !!')

DONE !!
